In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
pwd

In [ ]:
data_info = pd.read_csv("../input/lendingclub-data-sets/lending_club_info.csv",index_col='LoanStatNew')

In [ ]:
data_info.head()

In [ ]:
print(data_info.loc['revol_util']['Description'])

In [ ]:
def feat_info(col_name):
    print(data_info.loc[col_name]['Description'])

In [ ]:
feat_info('mort_acc')

In [ ]:
df = pd.read_csv('../input/lendingclub-data-sets/lending_club_loan_two.csv')

In [ ]:
df.head()

### EDA

In [ ]:
sns.countplot(x="loan_status",data=df);

In [ ]:
plt.figure(figsize=(10,6))
sns.distplot(df["loan_amnt"],kde=True,bins=50);

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(),annot=True,cmap ="viridis");
plt.ylim(10,0)

In [ ]:
feat_info("installment")

In [ ]:
feat_info("loan_amnt")

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x="installment",y="loan_amnt",data=df);

In [ ]:
sns.boxplot(x="loan_status",y="loan_amnt",data=df)

In [ ]:
df.groupby("loan_status")["loan_amnt"].describe()

In [ ]:
df["grade"].unique()

In [ ]:
df["sub_grade"].unique()

In [ ]:
sns.countplot(x="grade",data=df,hue="loan_status");

In [ ]:
plt.figure(figsize=(15,5))
subgrade_order = sorted(df["sub_grade"].unique())
sns.countplot(x="sub_grade",data=df,order=subgrade_order,palette="coolwarm");

In [ ]:
plt.figure(figsize=(15,5))
subgrade_order = sorted(df["sub_grade"].unique())
sns.countplot(x="sub_grade",data=df,order=subgrade_order,palette="coolwarm",hue="loan_status");

In [ ]:
f_and_g = df[(df["grade"]=="G") | (df["grade"]=="F")]

plt.figure(figsize=(15,5))
subgrade_order = sorted(f_and_g["sub_grade"].unique())
sns.countplot(x="sub_grade",data=f_and_g,order=subgrade_order,hue="loan_status");

In [ ]:
df["loan_repaid"] = df["loan_status"].map({"Fully Paid":1,"Charged Off":0})

In [ ]:
df.corr()["loan_repaid"].sort_values().drop("loan_repaid").plot(kind="bar");

### Data Manipulation

In [ ]:
df.describe().T

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
100 * df.isnull().sum() / len(df)

In [ ]:
df["emp_title"].nunique()

In [ ]:
df["emp_title"].value_counts()

In [ ]:
df.drop("emp_title",axis=1,inplace=True)

In [ ]:
sorted(df["emp_length"].dropna().unique())

In [ ]:
emp_length_order = ['< 1 year',
 '1 year',
 '2 years',
 '3 years',
 '4 years',
 '5 years',
 '6 years',
 '7 years',
 '8 years',
 '9 years',
 '10+ years'] 

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(x="emp_length",data=df,order=emp_length_order);

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(x="emp_length",data=df,order=emp_length_order,hue="loan_status");

In [ ]:
emp_co = df[df["loan_status"] == "Charged Off"].groupby("emp_length").count()["loan_status"]

In [ ]:
emp_fp = df[df["loan_status"] == "Fully Paid"].groupby("emp_length").count()["loan_status"]

In [ ]:
emp_len = emp_co/(emp_co+emp_fp)

In [ ]:
emp_len.plot(kind="bar");

* charge off rated are extremely similar across all employement lengths. So we drop the emp_lentgh column

In [ ]:
df.drop("emp_length",axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df["purpose"].head(10)

In [ ]:
df["title"].head(10)

In [ ]:
df.drop("title",axis=1,inplace=True)

In [ ]:
feat_info("mort_acc")

In [ ]:
df.corr()["mort_acc"].sort_values().drop("mort_acc").plot(kind="bar");

In [ ]:
total_acc_avg = df.groupby("total_acc").mean()["mort_acc"]

In [ ]:
def fill_mort_acc(total_acc,mort_acc):
    if np.isnan(mort_acc):
        return total_acc_avg[total_acc]
    else:
        return mort_acc

In [ ]:
df["mort_acc"] = df.apply(lambda x: fill_mort_acc(x["total_acc"],x["mort_acc"]),axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

### Categorical Variables and Dummy Variables

### term feature

In [ ]:
df.select_dtypes(["object"]).columns

In [ ]:
feat_info("term")

In [ ]:
df["term"].value_counts()

In [ ]:
df["term"] = df["term"].apply(lambda term: int(term[:3]))

### grade feature

In [ ]:
feat_info("grade")

In [ ]:
df["grade"].value_counts().sort_values(ascending=False)

In [ ]:
df.drop("grade",axis=1,inplace=True)

In [ ]:
dummies = pd.get_dummies(df["sub_grade"],drop_first=True)

In [ ]:
df = pd.concat([df.drop("sub_grade",axis=1),dummies],axis=1)

In [ ]:
df.columns

### verification_status, application_type, initial_list_status, purpose

In [ ]:
dummies = pd.get_dummies(df[["verification_status","application_type","initial_list_status","purpose"]],drop_first=True)
df = pd.concat([df.drop(["verification_status","application_type","initial_list_status","purpose"],axis=1),dummies],axis=1)

In [ ]:
df.head()

### home_ownership

In [ ]:
df["home_ownership"].value_counts()

In [ ]:
df["home_ownership"] = df["home_ownership"].replace(["NONE","ANY"],"OTHER")

In [ ]:
df["home_ownership"].value_counts()

In [ ]:
dummies = pd.get_dummies(df["home_ownership"],drop_first=True)
df = pd.concat([df.drop("home_ownership",axis=1),dummies],axis=1)

### address

In [ ]:
df["zip_code"] = df["address"].apply(lambda x:x[-5:])

In [ ]:
df["zip_code"].value_counts()

In [ ]:
dummies = pd.get_dummies(df["zip_code"],drop_first=True)
df = pd.concat([df.drop("zip_code",axis=1),dummies],axis=1)

In [ ]:
df.drop("address",axis=1,inplace=True)

### issue_d

* this would be data leakage, we wouldn't know beforehand whether or not a loan would be issued when using our model, so drop this feature

In [ ]:
feat_info("issue_d")

In [ ]:
df.drop("issue_d",axis=1,inplace=True)

### earliest_cr_line

* extract the year from this feature

In [ ]:
df["earliest_cr_year"] = df["earliest_cr_line"].apply(lambda x: int(x[-4:]))

In [ ]:
df.drop("earliest_cr_line",axis=1,inplace=True)

### Data Preprocessing

In [ ]:
df = df.drop("loan_status",axis=1)

In [ ]:
X = df.drop("loan_repaid",axis=1).values
y = df["loan_repaid"].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train.shape

### Creating Model

In [ ]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [ ]:
model = Sequential()

model.add(Dense(78,activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(39,activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(19,activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy",optimizer="adam")

In [ ]:
model.fit(x=X_train, y=y_train, epochs=10, batch_size=256, validation_data=(X_test,y_test))

### Save the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save("lending_club_model.h5")

### Evaluating Model Performance

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses.plot();

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
predictions = model.predict_classes(X_test)

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
confusion_matrix(y_test,predictions)

In [ ]:
import random
random.seed(101)
random_ind = random.randint(0,len(df))

new_customer = df.drop("loan_repaid",axis=1).iloc[random_ind]
new_customer

In [ ]:
new_customer = scaler.transform(new_customer.values.reshape(1,78))

In [ ]:
model.predict_classes(new_customer)

In [ ]:
df.iloc[random_ind]["loan_repaid"]